In [6]:
# ==============================================================
# MSc Project – Biometric Security Baseline + Adversarial Testing
# Author: Stella Williams
# Draft 3
# ==============================================================

# --------------------------------------------------------------
# Step 1: Initialise Model & Device
# --------------------------------------------------------------

import torch
import torch.nn as nn
from facenet_pytorch import InceptionResnetV1
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
import random, cv2
from PIL import Image
from tqdm import tqdm

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load pretrained FaceNet
model = InceptionResnetV1(pretrained="vggface2").eval().to(device)

# Loss & similarity
criterion = nn.CosineEmbeddingLoss()
cos = nn.CosineSimilarity(dim=1)


Using device: cpu


In [7]:
# --------------------------------------------------------------
# Step 2: Load and Filter LFW Dataset
# --------------------------------------------------------------

lfw_path = "/Users/stel/Documents/Dissertation/msc-biometric-security/Datasets/lfw-dataset"

transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

lfw_full = datasets.ImageFolder(root=lfw_path, transform=transform)
target_classes = random.sample(lfw_full.classes, 5)
print("Selected identities:", target_classes)

target_idx = [i for i, (_, label) in enumerate(lfw_full)
              if lfw_full.classes[label] in target_classes]

lfw_subset = Subset(lfw_full, target_idx)
lfw_loader = DataLoader(lfw_subset, batch_size=1, shuffle=True)


Selected identities: ['Alexandre_Daigle', 'Piotr_Anderszewski', 'Larry_Campbell', 'Francisco_Maturana', 'Carl_Levin']


In [8]:
# --------------------------------------------------------------
# Step 3: Attack and Defence Functions
# --------------------------------------------------------------

# --- FGSM ---
def fgsm_attack(image, grad, eps):
    return torch.clamp(image + eps * grad.sign(), 0, 1)

# --- PGD (strong attacker) ---
def pgd_attack(model, image, eps=0.4, alpha=0.02, steps=80):
    adv = image.clone().detach().to(device)
    adv.requires_grad = True

    for _ in range(steps):
        output = model(adv)
        loss = criterion(output, model(image).detach(), torch.tensor([1.0]).to(device))
        model.zero_grad()
        loss.backward()

        adv = adv + alpha * adv.grad.sign()
        adv = torch.clamp(image + torch.clamp(adv - image, -eps, eps), 0, 1).detach()
        adv.requires_grad = True

    return adv

# --- JPEG Defence ---
def jpeg_defence(img_tensor, quality=35):
    img_np = (img_tensor.detach().squeeze().permute(1,2,0).cpu().numpy()*255).astype(np.uint8)
    _, enc = cv2.imencode(".jpg", img_np, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
    dec = cv2.imdecode(enc, cv2.IMREAD_COLOR)
    rec = transforms.ToTensor()(Image.fromarray(cv2.cvtColor(dec, cv2.COLOR_BGR2RGB)))
    return rec.unsqueeze(0).to(device)

# --- Gaussian Blur ---
def blur_defence(img_tensor, kernel=5):
    img_np = (img_tensor.detach().squeeze().permute(1,2,0).cpu().numpy()*255).astype(np.uint8)
    blurred = cv2.GaussianBlur(img_np, (kernel, kernel), 0)
    rec = transforms.ToTensor()(Image.fromarray(cv2.cvtColor(blurred, cv2.COLOR_BGR2RGB)))
    return rec.unsqueeze(0).to(device)


In [9]:
# --------------------------------------------------------------
# Step 4: Visualisation Helper
# --------------------------------------------------------------

def show(img, title="Image"):
    plt.imshow(img.squeeze().permute(1,2,0).detach().cpu())
    plt.title(title)
    plt.axis("off")


In [10]:
# --------------------------------------------------------------
# Step 5: PGD Benchmark (Self-attack + Impersonation Attack)
# --------------------------------------------------------------

epsilons = [0.05, 0.1, 0.2, 0.3, 0.4]

sim_threshold_self = 0.99     # model stops recognising you
sim_threshold_imposter = 0.90 # model mistakenly thinks you're someone else

print("\n### PGD Dual Evaluation ###")

for eps in epsilons:
    self_fail = 0
    imposter_success = 0
    total = 0

    for _ in range(30):
        img, _ = next(iter(lfw_loader))
        img = img.to(device)
        embed_orig = model(img)

        # choose another random face to impersonate
        imp_img, _ = next(iter(lfw_loader))
        imp_img = imp_img.to(device)
        embed_imp = model(imp_img)

        # generate PGD adversarial
        adv = pgd_attack(model, img, eps=eps, alpha=0.02, steps=60)

        # SELF‑attack (does it stop matching itself?)
        sim_self = cos(embed_orig, model(adv)).item()
        if sim_self < sim_threshold_self: self_fail += 1

        # IMPERSONATION (does adv resemble target person?)
        sim_imp = cos(embed_imp, model(adv)).item()
        if sim_imp > sim_threshold_imposter: imposter_success += 1

        total += 1

    print(f"Epsilon {eps:.2f} | Self‑break: {self_fail/total:.2f} | Impersonation Success: {imposter_success/total:.2f}")



### PGD Dual Evaluation ###
Epsilon 0.05 | Self‑break: 0.00 | Impersonation Success: 0.20
Epsilon 0.10 | Self‑break: 0.00 | Impersonation Success: 0.27
Epsilon 0.20 | Self‑break: 0.00 | Impersonation Success: 0.07
Epsilon 0.30 | Self‑break: 0.00 | Impersonation Success: 0.17
Epsilon 0.40 | Self‑break: 0.00 | Impersonation Success: 0.13
